In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pprint


#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
dat = pd.read_csv('[urp] 전처리 한 데이터.csv')
dat.head()

,BIZ_NO,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부채총계,...,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,MDSCO_PRTC_YN,ESTB_GB,EMP_CNT,Industry,Stock_OX,신생기업여부,Target
0,6998800583,20220331,24965602.0,0.0,219000.0,0.0,25184602.0,1999913.0,0.0,1999913.0,...,1,N,N,N,1,2.0,K,(주),2,1
1,5168100635,20211231,615988.0,185615.0,42385.0,14120.0,658373.0,494966.0,103545.0,598511.0,...,1,N,Y,N,1,10.0,J,(주),2,1
2,1348715772,20211231,1596577.0,3802.0,353082.0,39340.0,1949659.0,771535.0,194663.0,966198.0,...,1,N,Y,N,1,14.0,J,(주),1,1
3,5158115284,20211231,30077990.0,6034264.0,15251977.0,7279856.0,45329967.0,15391552.0,5304808.0,20696360.0,...,1,N,Y,N,1,173.0,C,(주),1,1
4,1348631964,20211231,28431311.0,15498763.0,21334735.0,19795298.0,49766046.0,28142492.0,638761.0,28781253.0,...,1,N,Y,N,1,113.0,C,(주),1,1


In [ ]:
comp0 = pd.read_excel('액티브 중소법인 재무보유.xlsx')
comp1 = pd.read_excel('휴폐업 중소법인 재무보유 (2).xlsx')

comp0.head()
comp1.head()

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
0,1268145633,(주),용진테크,NaN,"Yongjin Tech Co., Ltd.",M,2,2,1,1,...,N,NaN,20000529.0,1,NaN,자동차부품 제조판매업,NaN,www.yongjintech.com,안상동,40.0
1,1388101749,NaN,신구건설,(주),"SHINKU C0NSTRUCTION CO.,LTD.",M,2,2,1,1,...,N,NaN,19850110.0,1,NaN,건축자재 판매업,NaN,www.shinku.co.kr,소재필,13.0
2,1068179240,(주),석정미디어,NaN,sung jung,M,2,2,1,1,...,N,NaN,19990601.0,1,NaN,"토너, 잉크, 프린터 외 컴퓨터 관련 주변기기",NaN,NaN,이상호,1.0
3,2248114507,(주),새한실업,NaN,"SAEHAN INDUSTRY CO.,LTD.",M,2,2,1,1,...,N,NaN,19980522.0,1,NaN,금속 캔 및 기타 포장용기 제조업,NaN,NaN,김정남외 1명,6.0
4,1168132785,(주),석영엔터프라이즈,NaN,"Seokyoung Enterprise Co.,Ltd.",M,2,2,1,1,...,N,NaN,19930226.0,1,NaN,스마트트레이 외,NaN,www.systeel.co.kr,박성진,40.0


In [ ]:
comp0.rename(columns={'CMP_NM1':'CMP_NM'}, inplace=True)
comp0.columns

Index(['BIZ_NO', 'CMP_PFIX_NM', 'CMP_NM', 'CMP_SFIX_NM', 'CMP_ENM', 'BZ_TYP',
       'CMP_SCL', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN',
       'VENT_YN', 'LIST_CD', 'IND_CD_ORDR', 'IND_CD1', 'IND_CD2', 'IND_CD3',
       'PB_ORG_TYP', 'MDSCO_PRTC_YN', 'HDOF_CMP_CD', 'ESTB_DATE', 'ESTB_GB',
       'LIST_DATE', 'MN_BIZ_CONT', 'NATN_NM', 'HOMEPAGE_URL', 'CEO_NM',
       'EMP_CNT'],
      dtype='object')

In [ ]:
comp = pd.concat([comp0[['BIZ_NO', 'CMP_NM']], comp1[['BIZ_NO', 'CMP_NM']]], axis = 0)
# comp1[['BIZ_NO', 'CMP_NM']]
comp.reset_index(drop = True, inplace = True)
comp

,BIZ_NO,CMP_NM
0,1138111739,한일가스산업
1,2078132193,엠피아이
2,1318167565,도영운수
3,5048128251,중원산업
4,1098164260,카타나골프
...,...,...
35716,4808501629,게방식당팩토리
35717,8018501578,우진푸드중부지점
35718,4118523419,윤월드푸드코리아다크써클브로스
35719,1738102533,기린테크


In [ ]:
merged = pd.merge(dat, comp, how='left', on='BIZ_NO')
merged

,BIZ_NO,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부채총계,...,FR_IVST_CORP_YN,VENT_YN,MDSCO_PRTC_YN,ESTB_GB,EMP_CNT,Industry,Stock_OX,신생기업여부,Target,CMP_NM
0,6998800583,20220331,24965602.0,0.0,219000.0,0.0,25184602.0,1999913.0,0.0,1999913.0,...,N,N,N,1,2.000000,K,(주),2,1,코크렙제40호기업구조조정부동산투자회사
1,5168100635,20211231,615988.0,185615.0,42385.0,14120.0,658373.0,494966.0,103545.0,598511.0,...,N,Y,N,1,10.000000,J,(주),2,1,태거스
2,1348715772,20211231,1596577.0,3802.0,353082.0,39340.0,1949659.0,771535.0,194663.0,966198.0,...,N,Y,N,1,14.000000,J,(주),1,1,코드마인드
3,5158115284,20211231,30077990.0,6034264.0,15251977.0,7279856.0,45329967.0,15391552.0,5304808.0,20696360.0,...,N,Y,N,1,173.000000,C,(주),1,1,원텍
4,1348631964,20211231,28431311.0,15498763.0,21334735.0,19795298.0,49766046.0,28142492.0,638761.0,28781253.0,...,N,Y,N,1,113.000000,C,(주),1,1,태성
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35762,3088512085,20180630,13351633.0,0.0,19836205.0,0.0,33187838.0,11338737.0,9549530.0,20888267.0,...,N,N,N,4,67.000000,C,(유),1,0,삼각에프엠씨웰빙랜드
35763,6108508510,20180331,5409774.0,943751.0,4856884.0,3764094.0,10266658.0,620470.0,1500973.0,2121443.0,...,N,N,N,1,22.000000,M,(주),1,0,대운가스프랜트
35764,6028522170,20180331,2196004.0,990945.0,174426.0,90816.0,2370430.0,6952508.0,794755.0,7747262.0,...,N,N,N,1,35.000000,I,(주),1,0,크라운에셋크라운하버호텔
35765,6088511628,20180331,2196004.0,990945.0,174426.0,90816.0,2370430.0,6952508.0,794755.0,7747262.0,...,N,N,N,1,4.000000,L,미분류,1,0,크라운에셋


In [ ]:
com = merged['CMP_NM']



In [ ]:
def news_cont1(lin):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}
    try:
        news_contents =[]
        if "news.naver.com" in lin:
            #각 기사 html get하기
            news = requests.get(lin,headers=headers)
            news_html = BeautifulSoup(news.text,"html.parser")

            # 뉴스 본문 가져오기
            content = news_html.select("div#dic_area")
            if content == []:
                content = news_html.select("#articeBody")

            # 기사 텍스트만 가져오기
            content = ''.join(str(content))

            # html태그제거 및 텍스트 다듬기
            pattern1 = '<[^>]*>'
            content = re.sub(pattern=pattern1, repl='', string=content)
            pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
            content = content.replace(pattern2, '')
            content = re.sub("\n","",content)

            news_contents.append(content)
            
        else:
            news_contents.appned('기사를 불러올 수 없습니다.')
    except:
        pass

    return news_contents
        
        

In [ ]:
# 함수정의 - 네이버 뉴스api를 이용한 기업별 뉴스정보 수집

def crwaler(i) : 
    client_id = "R1RXO1IVS_PvTLY9sp_F"
    client_secret = "YbFf1dQxRh"
    
    search_word = i #검색어
    encode_type = 'json' 
    max_display = 100 #출력 뉴스 수
    sort = 'date' #결과값의 정렬기준 시간순 date, 관련도 순 sim
    start = 1 # 출력 위치

    url = f"https://openapi.naver.com/v1/search/news.{encode_type}?query={search_word}&display={str(int(max_display))}&start={str(int(start))}&sort={sort}"

    #헤더에 아이디와 키 정보 넣기
    headers = {'X-Naver-Client-Id' : client_id,
               'X-Naver-Client-Secret':client_secret
               }

    #HTTP요청 보내기
    r = requests.get(url, headers=headers)
    #요청 결과 보기 200 이면 정상적으로 요청 완료
    j = r.json()
    
    while (j['display']!= 0) :
        df = pd.DataFrame(j['items'])

        def clean_html(x):
          x = re.sub("\&\w*\;","",x)
          x = re.sub("<.*?>","",x)
          return x
        
        lin = list(map(lambda x: list(x.values())[2], j['items']))
        content = list(map(lambda x: list(news_cont1(x)), lin))

        df['company'] = i
        df['title'] = df['title'].apply(lambda x: clean_html(x))
        df['description'] = df['description'].apply(lambda x: clean_html(x))
        df['contents'] = content
        

        df.to_csv(f'news_search_result_{search_word}.csv', index=False)
        print(f'{search_word}기업 저장 완료')
        break
    else : 
        print(f'{search_word}기업 뉴스 없음')

In [ ]:
for i in com:
    crwaler(i)

코크렙제40호기업구조조정부동산투자회사기업 뉴스 없음
태거스기업 저장 완료
코드마인드기업 저장 완료
원텍기업 저장 완료
태성기업 저장 완료
트리플기업 저장 완료
비비드스튜디오기업 저장 완료
밸로인기업 저장 완료
로지올기업 저장 완료
아이엠에이플러스기업 저장 완료
제이텍기업 저장 완료
케이에프기업 저장 완료
제이에이알시스템즈기업 저장 완료
도광건설기업 저장 완료
미건기업 저장 완료
미건기업 저장 완료
에듀십컨설팅기업 뉴스 없음
선진레미콘기업 저장 완료
베올리아코리아에너지기업 저장 완료
금천지에스기업 저장 완료
하이딥기업 저장 완료
케이원목재기업 저장 완료
국기기업 저장 완료
화성이엔지기업 저장 완료
구암종합건설기업 저장 완료
원산기업 저장 완료
어썸스토리기업 저장 완료
탑앤씨기업 저장 완료
하나글로벌인재개발원피에프브이기업 저장 완료
다남건설기업 저장 완료
모비데이즈기업 저장 완료
판교에스디투기업 저장 완료
양심모터스기업 저장 완료
베셀기업 저장 완료
나노글라스기업 저장 완료
나노글라스기업 저장 완료
요산요수기업 저장 완료
지나송기업 저장 완료
쥬라리움기업 저장 완료
테라크라우드대부기업 저장 완료
한가온종합건설기업 저장 완료
코닉오토메이션기업 저장 완료
비아로사기업 저장 완료
리더스기업 저장 완료
파이버프로기업 저장 완료
에이프로젠기업 저장 완료
토경기업 저장 완료
삼주기업 저장 완료
이너라인코퍼레이션기업 저장 완료
클래스마켓기업 저장 완료
이강건설기업 저장 완료
하루티앤티기업 저장 완료
한국인베스트먼트뱅크기업 저장 완료
세진건설기업 저장 완료
티제이건설기업 저장 완료
쌤빛기업 저장 완료
한동씨앤씨기업 저장 완료
패스트비코리아기업 저장 완료
새롬석면환경기업 저장 완료
소우케이티지푸드기업 뉴스 없음
에스에프피씨기업 뉴스 없음
기린테크기업 저장 완료
네오젠소프트기업 저장 완료
대경에스티기업 저장 완료
야누스랩스기업 저장 완료
경향셀기업 저장 완료
팩토리얼홀딩스기업 저장 완료
에이밍하이제십일차기업 뉴스 없음
타마스기업 저장 완료
성화건설기업 저장 완료
엘티시스템기업 저장

KeyError: 'title'